In [1]:
!pip install brainflow
!pip install matplotlib
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from brainflow.data_filter import DataFilter, FilterTypes
import numpy as np
import matplotlib.pyplot as plt
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 56.3 MB/s eta 0:00:00


In [2]:
# imports
from IPython.display import Audio, display
import librosa as li
import soundfile as sf
import numpy as np
import requests
import torch
torch.set_grad_enabled(False)

!pip install git+https://github.com/acids-ircam/RAVE.git


  Cloning https://github.com/acids-ircam/RAVE.git to /tmp/pip-req-build-v6828x0r
  Running command git clone --filter=blob:none --quiet https://github.com/acids-ircam/RAVE.git /tmp/pip-req-build-v6828x0r
  Resolved https://github.com/acids-ircam/RAVE.git to commit f048ec4569afba7c6ba38d590be8a07b8ab24840
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/

In [3]:
def download_file(url, file_path):
    """
    Download file from a given URL and save it to the specified file path.
    """
    response = requests.get(url)
    response.raise_for_status()  # This will raise an exception if there is an error

    with open(file_path, 'wb') as file:
        file.write(response.content)
# download an audio file
url = 'https://ccrma.stanford.edu/~jos/wav/gtr-nylon22.wav'
file_path = 'audio.wav'
download_file(url, file_path)

x, sr = li.load('audio.wav',sr=44100)

display(Audio(x, rate=sr))

In [4]:
import torch
from huggingface_hub import hf_hub_download

"""
Musical Instruments

guitar_iil_b2048_r48000_z16.ts — electric guitar timbre. 

sax_soprano_franziskaschroeder_b2048_r48000_z20.ts — soprano sax improvisation. 

organ_archive_b2048_r48000_z16.ts — church/organ archive. 

organ_bach_b2048_sr48000_z16.ts — Bach organ recordings. 

mrp_strengjavera_b2048_r44100_z16.ts — magnetic resonator piano. 


Voice / Singing / Speech

voice_vocalset_b2048_r48000_z16.ts — singing voice dataset VocalsSet. 

voice_hifitts_b2048_r48000_z16.ts — Hi-Fi TTS audiobooks dataset. 

voice_jvs_b2048_r44100_z16.ts — single speaker from JVS corpus. 

voice_vctk_b2048_r44100_z22.ts — multispeaker VCTK corpus. 

voice_multivoice_b2048_r48000_z11.ts — combined multiple voice/singing datasets. 


Birds / Nature / Marine Sounds

birds_motherbird_b2048_r48000_z16.ts — bird sounds (Motherbird piece). 

birds_pluma_b2048_r48000_z12.ts — bird sounds (Pluma instrument). 

water_pondbrain_b2048_r48000_z16.ts — water recordings for “Pond Brain”. 

humpbacks_pondbrain_b2048_r48000_z20.ts — humpback whale recordings. 

marinemammals_pondbrain_b2048_r48000_z20.ts — marine mammals. 


Other / Miscellaneous

thales_magnets_b2048_r48000_z8.ts — metal/wood scratch magnet sounds (Thales). 

birds_dawnchorus_b2048_r48000_z8.ts — dawn chorus bird recordings. 
"""

# 1. Download a RAVE model (example: guitar)
checkpoint_path = hf_hub_download(
    repo_id="Intelligent-Instruments-Lab/rave-models",
    filename="guitar_iil_b2048_r48000_z16.ts"
)

# 2. Load the TorchScript model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.jit.load(checkpoint_path, map_location=device)
model.eval()

print("Model loaded successfully on", device)


guitar_iil_b2048_r48000_z16.ts:   0%|          | 0.00/164M [00:00<?, ?B/s]

Model loaded successfully on cuda


## Implementing Brainflow

In [5]:
PARAMS = BrainFlowInputParams()
PARAMS.serial_port = "/dev/cu.usbserial-DP04VZSH"

In [6]:
try:
    board_ID = BoardIds.CYTON_DAISY_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to Cyton+Daisy board.")
except Exception as e:
    print(f"Error connecting to Cyton+Daisy board: {e}")
    input("Want to use synthetic board instead? Press Enter to continue...")
    board_ID = BoardIds.SYNTHETIC_BOARD
    board = BoardShim(board_ID, PARAMS)
    board.prepare_session()
    print("Successfully connected to synthetic board.")

board.release_session()

Error connecting to Cyton+Daisy board: UNABLE_TO_OPEN_PORT_ERROR:2 unable to prepare streaming session


[2025-11-03 20:47:38.824] [board_logger] [info] incoming json: {
    "file": "",
    "file_anc": "",
    "file_aux": "",
    "ip_address": "",
    "ip_address_anc": "",
    "ip_address_aux": "",
    "ip_port": 0,
    "ip_port_anc": 0,
    "ip_port_aux": 0,
    "ip_protocol": 0,
    "mac_address": "",
    "master_board": -100,
    "other_info": "",
    "serial_number": "",
    "serial_port": "/dev/cu.usbserial-DP04VZSH",
    "timeout": 0
}
[2025-11-03 20:47:38.824] [board_logger] [info] opening port /dev/cu.usbserial-DP04VZSH
[2025-11-03 20:47:38.824] [board_logger] [error] Make sure you provided correct port name and have permissions to open it(run with sudo/admin). Also, close all other apps using this port.


StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

In [ ]:
# download rave parameters/weights and build the model
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/percussion'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/vintage'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/nasa'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/darbouka_onnx'
# url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/VCTK'
url = 'https://play.forum.ircam.fr/rave-vst-api/get_model/musicnet'
# you can learn more about each model at:
# https://acids-ircam.github.io/rave_models_download
file_path = 'model.ts'
download_file(url, file_path)

model = torch.jit.load("model.ts").eval()

In [ ]:
x, sr = li.load('audio.wav',sr=44100)
x = torch.from_numpy(x).reshape(1,1,-1)

# generate audio from random numbers
z = model.encode(x)
z = torch.from_numpy(np.random.randn(*z.shape).astype(np.float32))
x_hat = model.decode(z).numpy().reshape(-1)

sf.write("model_output.wav", x_hat, sr)
display(Audio(x_hat, rate=sr))


# Music Generation Workflow

### Audio → Encoder → Latent z → Decoder → Audio


In [ ]:
from brainflow.data_filter import DataFilter, AggOperations, DetrendOperations, WaveletDenoisingTypes

In [ ]:
def preprocess(eeg_data):
    # bandpass parameters
    highpass = 0.1
    lowpass = 50

    # preprocessed final
    preprocessed_data = []

    for channel_data in eeg_data:
        # detrend
        DataFilter.detrend(channel_data, DetrendOperations.LINEAR.value)
        
        # band pass application
        DataFilter.perform_bandpass(
            channel_data,
            250, #sampling rate
            (lowpass + highpass) / 2, #central freq
            highpass - lowpass, #band width
            4,#order
            FilterTypes.BUTTERWORTH_ZERO_PHASE, #filter type
            0 #ripple
        )
        
        # eliminate notch filter ~60Hz
        DataFilter.remove_environmental_noise(channel_data, 
                                              250, #sampling rate
                                              NoiseTypes.SIXTY) #noise type
        
        # downsample using Nyquists theorem (half it)
        downsampled = DataFilter.perform_downsampling(
            channel_data, 
            2, # period
            AggOperations.MEAN.value #operation
        )

        # normalize for stable latent mapping
        downsampled = (downsampled - np.mean(downsampled)) / (np.std(downsampled) + 1e-8)
        preprocessed_data.append(downsampled)
    
    return np.array(preprocessed_data)

"""
eeg = preprocess(eeg_data)  
eeg = np.expand_dims(eeg, 0)  # add batch dimension
z = torch.from_numpy(eeg).float().to(device)
"""

In [ ]:
import numpy as np
from brainflow.data_filter import (
    DataFilter, FilterTypes, AggOperations, NoiseTypes,
    DetrendOperations, WaveletDenoisingTypes
)

import pywt
import numpy as np

def wavelet_denoise(channel_data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    uthresh = sigma * np.sqrt(2 * np.log(len(channel_data)))
    coeffs[1:] = [pywt.threshold(c, value=uthresh, mode='soft') for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)


def preprocess_eeg_for_rave(eeg_data, sampling_rate=250):
    """
    Preprocess EEG data shaped as [channels, samples].
    Returns clean array of same structure.
    """
    highpass = 0.1
    lowpass = 50
    preprocessed_data = []

    for channel_data in eeg_data:
        channel_data = np.copy(channel_data)  # ensure writable

        # Detrend
        DataFilter.detrend(channel_data, DetrendOperations.LINEAR.value)

        # Bandpass
        DataFilter.perform_bandpass(
            channel_data,
            sampling_rate,
            (lowpass + highpass) / 2,
            lowpass - highpass,
            4,
            FilterTypes.BUTTERWORTH_ZERO_PHASE,
            0
        )

        # Notch (60 Hz)
        DataFilter.remove_environmental_noise(
            channel_data,
            sampling_rate,
            NoiseTypes.SIXTY
        )

        # Wavelet denoising
        denoised = wavelet_denoise(channel_data)

        # Downsample
        downsampled = DataFilter.perform_downsampling(
            channel_data,
            2,
            AggOperations.MEAN.value
        )

        # Normalize
        downsampled = (downsampled - np.mean(downsampled)) / (np.std(downsampled) + 1e-8)

        preprocessed_data.append(downsampled)

    return np.array(preprocessed_data)


In [ ]:
import numpy as np
import torch
from scipy.signal import welch

def bandpower(data, sf, band, window_sec=2):
    """
    Compute average power of EEG band.
    data: (channels, samples)
    band: [low, high]
    """
    low, high = band
    freqs, psd = welch(data, sf, nperseg=int(window_sec * sf))
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    return np.mean(psd[:, idx_band], axis=1)

def eeg_to_latent(eeg_preprocessed, sf=125, latent_dim=16, frames=64):
    """
    Map EEG to RAVE latent tensor using both raw rhythmic data
    and bandpower-derived emotional states.
    """
    # Compute bandpowers (alpha, beta, theta)
    alpha = bandpower(eeg_preprocessed, sf, [8, 12])
    beta  = bandpower(eeg_preprocessed, sf, [13, 30])
    theta = bandpower(eeg_preprocessed, sf, [4, 8])

    # Normalize each
    alpha_norm = (alpha - np.mean(alpha)) / (np.std(alpha) + 1e-8)
    beta_norm  = (beta  - np.mean(beta))  / (np.std(beta) + 1e-8)
    theta_norm = (theta - np.mean(theta)) / (np.std(theta) + 1e-8)

    # Combine rhythmic signal (time-varying) with global state (bandpower)
    # Downsample or interpolate EEG to match RAVE frame count
    eeg_resampled = np.interp(
        np.linspace(0, eeg_preprocessed.shape[1], frames),
        np.arange(eeg_preprocessed.shape[1]),
        np.mean(eeg_preprocessed, axis=0)
    )

    # Stack into latent tensor (1, latent_dim, frames)
    z = np.zeros((1, latent_dim, frames), dtype=np.float32)

    # Use first few dimensions to encode emotional/EEG features
    z[0,0,:] = eeg_resampled  # overall EEG rhythm
    z[0,1,:] = alpha_norm.mean()  # calmness / relaxation
    z[0,2,:] = beta_norm.mean()   # focus / excitement
    z[0,3,:] = theta_norm.mean()  # meditation / drowsiness

    # The rest can hold small jitter for richness
    z[0,4:,:] = np.random.randn(latent_dim-4, frames) * 0.05

    return torch.from_numpy(z).float()


In [ ]:
# getting data
print("starting stream...")
board.prepare_session()
board.start_stream()

# get 10 second of data to process
time.sleep(10)  # collect data for 10 seconds
starting_data = board.get_board_data()

board.stop_stream()
board.release_session()

EEG_CHANNELS = BoardShim.get_eeg_channels(board_ID)
eeg_data = starting_data[EEG_CHANNELS]

In [ ]:
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, DetrendOperations, WaveletDenoisingTypes

preprocessed = preprocess_eeg_for_rave(eeg_data)

In [ ]:
# Assume you already have 'preprocessed' EEG from your function
z = eeg_to_latent(preprocessed, sf=125, latent_dim=16, frames=64).to(device)
x_hat = model.decode(z).detach().cpu().numpy().reshape(-1)

sf.write("eeg_rave_output.wav", x_hat, 44100)
display(Audio(x_hat, rate=44100))


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # move entire model to GPU
z = eeg_to_latent(preprocessed, sf=125, latent_dim=16, frames=64).to(device)
x_hat = model.decode(z).detach().cpu().numpy().reshape(-1)

sf.write("eeg_rave_output.wav", x_hat, 44100)
display(Audio(x_hat, rate=44100))
